### Exercise 6

In [34]:
import numpy as np
from math import floor,factorial
from scipy import stats

In [12]:
floor(5.1)

5

In [13]:
m=10
lambd = 1
s=8
A= lambd*s

In [ ]:
def 

In [14]:
def g(i):
    if (i > m): return 0
    if (i < 0): return 0
    return (A**i)/factorial(i)

In [125]:
def sample_MCMC(sample_size):
    Xi=0
    sample = np.empty( 0 , dtype = 'float64' )
    sample = np.append( sample , Xi )
    for i in range(sample_size-1):
        Dxi= floor(3* np.random.normal(0,1,1)[0])
        Yi = Xi + Dxi
        acceptance = 'reject'
        if (g(Yi)>=g(Xi)):
            sample = np.append( sample , Yi )
            Xi=Yi
            continue
        if (np.random.uniform(0,1,1)[0]<=(g(Yi)/g(Xi))):
            sample = np.append( sample , Yi )
            Xi=Yi
            continue
        sample = np.append( sample , Xi )
    return sample
            
            

In [126]:
P = [g(i) for i in range(m)]
denominator = sum(P)
P = [i/denominator for i in (P) ]
X = sample_MCMC(10000)
sim_sum = sum(X)
Observ, bins = np.histogram(X, bins= m)
Observed = [i/sim_sum for i in Observ]
Expected = P
least_squares = [(Obs-Exp)**2/Exp for Obs, Exp in list(zip(Expected,Observed))]
chi_sqare_calculated_value = sum(least_squares)
p_value = stats.chi2.sf(chi_sqare_calculated_value, m)
p_value

0.8931724516949544

In [101]:
def chisquare(obs, no_class):
    Observed, bins = np.histogram(obs, bins= no_class)
    expct =  len(obs)/no_class
    T = 0
    for i in Observed:
        xx = ((i-expct)**2)/expct
        T+= xx
    p = chi2.pdf(T,9)
    print('p-value is:',p)
    if(p <= 0.05):
        print('The generated random numbers are not uniformly distributed')
    else:
        print('The generated random numbers are uniformly distributed')

In [116]:
def sample_MCMC(sample_size):
    Xi=0
    sample = np.empty( 0 , dtype = 'float64' )
    sample = np.append( sample , Xi )
    for i in range(sample_size-1):
        Dxi= floor(10 * np.random.uniform(-1,1,1)[0])
        Yi = Xi + Dxi
        acceptance = 'reject'
        if (g(Yi)>=g(Xi)):
            sample = np.append( sample , Yi )
            Xi=Yi
            continue
        if (np.random.uniform(0,1,1)[0]<=(g(Yi)/g(Xi))):
            sample = np.append( sample , Yi )
            Xi=Yi
            continue
        sample = np.append( sample , Xi )
    return sample

In [117]:
P = [g(i) for i in range(m)]
denominator = sum(P)
P = [i/denominator for i in (P) ]
X = sample_MCMC(10000)
sim_sum = sum(X)
Observ, bins = np.histogram(X, bins= m)
Observed = [i/sim_sum for i in Observ]
Expected = P
least_squares = [(Obs-Exp)**2/Exp for Obs, Exp in list(zip(Expected,Observed))]
chi_sqare_calculated_value = sum(least_squares)
p_value = stats.chi2.sf(chi_sqare_calculated_value, m)
p_value

[1.0, 8.0, 32.0, 85.33333333333333, 170.66666666666666, 273.06666666666666, 364.0888888888889, 416.1015873015873, 416.1015873015873, 369.86807760141096]
[   2   16  140  318  622 1131 1506 1768 1793 2704]


0.8486249611100332

## Part 2

### a 

In [236]:
def g_2d(x):
    i,j = x
    if (i >= m): return 0
    if (j >= m): return 0
    if (i+j > m): return 0
    if (i < 0): return 0
    if (j < 0): return 0
    if (i+j < 0): return 0
    return ((A**i)/factorial(i))*((A**j)/factorial(j))

In [259]:
floor(-1.01)

-2

In [237]:
def sample_MCMC_2d(sample_size):
    Xi=(0,0)
    sample = []
    sample.append( Xi )
    for i in range(sample_size-1):
        #print('sample=',sample)
        Dxi_1= floor(3* np.random.normal(0,1,1)[0])
        Dxi_2= floor(3* np.random.normal(0,1,1)[0])
        Yi = (Xi[1] + Dxi_1,Xi[1]+Dxi_2)
        acceptance = 'reject'
        #print('Xi=',Xi)
        #print('Yi=',Yi)
        #print('g(Xi)=',g(Xi))
        #print('g(Yi)=',g(Yi))
        #input()
        if (g_2d(Yi)>=g_2d(Xi)):
            sample.append( Yi )
            Xi=Yi
            continue
        if (np.random.uniform(0,1,1)[0]<=(g_2d(Yi)/g_2d(Xi))):
            sample.append(  Yi )
            Xi=Yi
            continue
        sample.append(  Xi )
    return sample

In [238]:
probs = np.zeros((m, m), dtype='float64')


In [239]:
for i in range(m):
    for j in range(i,m):
        probs[i,j] = g_2d((i,j))

In [240]:
P = probs
denominator = sum(P)
P = P/denominator

In [247]:
counts =  np.zeros((m, m), dtype='float64')
X = sample_MCMC_2d(10000)

for element in X:
    #print(element)
    i,j = element
    counts[i,j] += 1
sim_sum = sum(counts)
Observed = counts/sim_sum
Expected = P
least_squares = np.zeros((m, m), dtype='float64')
for i in range(m):
    for j in range(i,m):
        if (Expected[i,j] == 0.0): 
            least_squares[i,j]=0
            continue
        least_squares[i,j] = (Observed[i,j]-Expected[i,j])**2/Expected[i,j]
chi_sqare_calculated_value = sum(sum(least_squares))
p_value = stats.chi2.sf(chi_sqare_calculated_value, m)
p_value

4.62121197294037


0.9150045623977796

In [257]:
def sample_MCMC_2d_dwise(sample_size):
    Xi=(0,0)
    sample = []
    sample.append( Xi )
    for i in range(sample_size-1):
        #print('sample=',sample)
        Dxi_1= floor(3* np.random.normal(0,1,1)[0])
        Dxi_2= floor(3* np.random.normal(0,1,1)[0])
        Yi_1d = (Xi[1] + Dxi_1,Xi[1])
        #print('Xi=',Xi)
        #print('Yi=',Yi)
        #print('g(Xi)=',g(Xi))
        #print('g(Yi)=',g(Yi))
        #input()
        if (g_2d(Yi_1d)>=g_2d(Xi)):
            Xi=Yi_1d
         
        elif (np.random.uniform(0,1,1)[0]<=(g_2d(Yi_1d)/g_2d(Xi))):
            Xi=Yi_1d
        Yi_2d = (Xi[1],Xi[1] + Dxi_2)
        
        
        if (g_2d(Yi_2d)>=g_2d(Xi)):
            sample.append(Yi_2d )
            Xi=Yi_2d
            continue
        if (np.random.uniform(0,1,1)[0]<=(g_2d(Yi_2d)/g_2d(Xi))):
            sample.append(  Yi_2d )
            Xi=Yi_2d
            continue
        
        sample.append(  Xi )
    return sample

In [258]:
counts =  np.zeros((m, m), dtype='float64')
X = sample_MCMC_2d_dwise(10000)

for element in X:
    #print(element)
    i,j = element
    counts[i,j] += 1
sim_sum = sum(counts)
Observed = counts/sim_sum
Expected = P
least_squares = np.zeros((m, m), dtype='float64')
for i in range(m):
    for j in range(i,m):
        if (Expected[i,j] == 0.0): 
            least_squares[i,j]=0
            continue
        least_squares[i,j] = (Observed[i,j]-Expected[i,j])**2/Expected[i,j]
chi_sqare_calculated_value = sum(sum(least_squares))
p_value = stats.chi2.sf(chi_sqare_calculated_value, m)
p_value

0.9502391486142272